# Multi Agente usando ADK

## Configurando o ambiente

In [1]:
!pip install google-adk -q
!pip install litellm -q

print("Installation complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 12.6 MB/s eta 0:00:00
Installation complete.


In [2]:
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [3]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Anthropic API Key set: {'Yes' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

API Keys Set:
Google API Key set: Yes
OpenAI API Key set: Yes
Anthropic API Key set: Yes


In [4]:
MODEL_GEMINI_2_5_FLASH = "gemini-2.5-flash"
MODEL_GPT_4O = "openai/gpt-4.1-nano"
MODEL_CLAUDE_SONNET = "anthropic/claude-sonnet-4-5-20250929"

print("\nEnvironment configured.")


Environment configured.


## O Primeiro Agente
O tutorial traz a ferramenta como uma função python, limitada e repetida de outra atividade.
Implementarei uma função usando a plataforma open-meteo <https://open-meteo.com/en/docs>


In [25]:
# @title Ferramenta do Agente (open-meteo API)
import requests

def get_weather_open_meteo(city_name: str):
    """
    Busca o clima atual para uma cidade pelo nome, usando Open-Meteo
    """
    try:
        # Geocoding (Converter nome da cidade em Lat/Long)
        geo_url = "https://geocoding-api.open-meteo.com/v1/search"
        geo_params = {"name": city_name, "count": 1, "language": "pt", "format": "json"}

        geo_res = requests.get(geo_url, params=geo_params).json()

        if not geo_res.get("results"):
            return f"Não encontrei a localização: {city_name}"

        location = geo_res["results"][0]
        lat = location["latitude"]
        long = location["longitude"]
        real_name = location["name"]

        # Busca Meteorologica (Usando as coordenadas)
        meteo_url = "https://api.open-meteo.com/v1/forecast"
        meteo_params = {
            "latitude": lat,
            "longitude": long,
            "current": "temperature_2m,relative_humidity_2m,wind_speed_10m"
        }

        meteo_res = requests.get(meteo_url, params=meteo_params).json()
        current = meteo_res.get("current", {})

        return (f"Previsão para {real_name}: "
                f"Temperatura de {current.get('temperature_2m')}°C, "
                f"Umidade de {current.get('relative_humidity_2m')}%, "
                f"Vento a {current.get('wind_speed_10m')} km/h.")

    except Exception as e:
        return f"Erro ao processar o pedido: {e}"


In [26]:
print(get_weather_open_meteo("xique-xique"))

Previsão para Xique-Xique: Temperatura de 29.1°C, Umidade de 48%, Vento a 14.7 km/h.


In [13]:
# @title Agente Meteorologico
AGENT_MODEL = MODEL_CLAUDE_SONNET

weather_agent = Agent(
    name="agente_meteorologico_v1",
    model=AGENT_MODEL,
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather_open_meteo],
)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

Agent 'agente_meteorologico_v1' created using model 'anthropic/claude-sonnet-4-5-20250929'.


In [14]:
# @title Configurando Session Service e Runner

# --- Session Service ---
# SessionService armaena estato e histórico de conversação
# InMemorySessionService é um armazenamento temporario
session_service = InMemorySessionService()

# constantes para identificação do contexto
APP_NAME = "agente_meteorologico_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # id fixo para simplificar

# cria uma sessão especifica onde ocorerrá a conversação
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Sessão criada: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Runner orquestra o loop de execução do agente
runner = Runner(
    agent=weather_agent, # o agente para executar
    app_name=APP_NAME,   # associa as execução ao app
    session_service=session_service # usa o gerenciado de sessão criado acima
)
print(f"Runner criado para o agente '{runner.agent.name}'.")

Sessão criada: App='agente_meteorologico_app', User='user_1', Session='session_001'
Runner criado para o agente 'agente_meteorologico_v1'.


In [15]:
# @title Definindo a Função de Interação com o Agente

from google.genai import types

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> Questão do usuário: {query}")

  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # logica do agente executada de forma assíncrona e produz eventos.
  # a interação é feita através dos eventos até a resposta final.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # motra todos os eventos durante a execução
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      if event.is_final_response():
          if event.content and event.content.parts:
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          break # Stop processing events once the final response is found

  print(f"<<< Resposta do Agente: {final_response_text}")

In [ ]:
# @title Executando a conversa com o agente

async def run_conversation():
    await call_agent_async("Como está o clima em Brasília?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)
    """
    await call_agent_async("E lá em Maceió?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("Conte-me como esta o clima em Curitiba?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)
    """
await run_conversation()

# --- OR ---

# Uncomment the following lines if running as a standard Python script (.py file):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"An error occurred: {e}")


>>> Questão do usuário: Como está o clima em Palmas?
<<< Resposta do Agente: O clima em **Palmas** está assim:

🌡️ **Temperatura:** 29.6°C  
💧 **Umidade:** 64%  
💨 **Vento:** 4.0 km/h  

O clima está agradável, com temperatura quente típica da região e vento leve!

>>> Questão do usuário: E lá em Maceió?
<<< Resposta do Agente: O clima em **Maceió** está assim:

🌡️ **Temperatura:** 27.0°C  
💧 **Umidade:** 76%  
💨 **Vento:** 16.4 km/h  

O clima está agradável com temperatura amena, umidade mais alta (típico do litoral) e vento moderado, perfeito para aproveitar as praias! 🏖️

>>> Questão do usuário: Conte-me como esta o clima em Curitiba?
<<< Resposta do Agente: O clima em **Curitiba** está assim:

🌡️ **Temperatura:** 23.4°C  
💧 **Umidade:** 77%  
💨 **Vento:** 2.5 km/h  

O clima está bastante agradável e ameno, com temperatura confortável, umidade relativamente alta e vento bem fraco. Um dia tranquilo na capital paranaense! 🌤️


In [16]:
# @title Criando um Agente com o ChatGPT

# --- Agent using GPT-4.1-nano ---
weather_agent_gpt = None
runner_gpt = None

try:
    weather_agent_gpt = Agent(
        name="weather_agent_gpt",
        model=LiteLlm(model=MODEL_GPT_4O),
        description="Provides weather information (using GPT-4o).",
        instruction="You are a helpful weather assistant powered by GPT-4o. "
                    "Use the 'get_weather' tool for city weather requests. "
                    "Clearly present successful reports or polite error messages based on the tool's output status.",
        tools=[get_weather_open_meteo],
    )
    print(f"Agent '{weather_agent_gpt.name}' created using model '{MODEL_GPT_4O}'.")

    session_service_gpt = InMemorySessionService()

    APP_NAME_GPT = "weather_tutorial_app_gpt"
    USER_ID_GPT = "user_1_gpt"
    SESSION_ID_GPT = "session_001_gpt"

    session_gpt = await session_service_gpt.create_session(
        app_name=APP_NAME_GPT,
        user_id=USER_ID_GPT,
        session_id=SESSION_ID_GPT
    )
    print(f"Sessão criada: App='{APP_NAME_GPT}', Usuário='{USER_ID_GPT}', Sessão='{SESSION_ID_GPT}'")

    runner_gpt = Runner(
        agent=weather_agent_gpt,
        app_name=APP_NAME_GPT,
        session_service=session_service_gpt
        )
    print(f"Runner criado '{runner_gpt.agent.name}'.")

    print("\n--- Testing GPT Agent ---")
    await call_agent_async(query = "Como está o clima em Tóquio?",
                           runner=runner_gpt,
                           user_id=USER_ID_GPT,
                           session_id=SESSION_ID_GPT)
    # --- OR ---

    # Uncomment the following lines if running as a standard Python script (.py file):
    # import asyncio
    # if __name__ == "__main__":
    #     try:
    #         asyncio.run(call_agent_async(query = "What's the weather in Tokyo?",
    #                      runner=runner_gpt,
    #                       user_id=USER_ID_GPT,
    #                       session_id=SESSION_ID_GPT)
    #     except Exception as e:
    #         print(f"An error occurred: {e}")

except Exception as e:
    print(f"❌ Could not create or run GPT agent '{MODEL_GPT_4O}'. Check API Key and model name. Error: {e}")


Agent 'weather_agent_gpt' created using model 'openai/gpt-4.1-nano'.
Sessão criada: App='weather_tutorial_app_gpt', Usuário='user_1_gpt', Sessão='session_001_gpt'
Runner criado 'weather_agent_gpt'.

--- Testing GPT Agent ---

>>> Questão do usuário: Como está o clima em Tóquio?
<<< Resposta do Agente: O clima em Tóquio está com uma temperatura de cerca de 6°C, umidade de 74% e vento a aproximadamente 4 km/h.


## Implementando um Time de Agentes (c/ delegação automática)

In [8]:
# @title Definindo ferramentas dos sub-agentes
from typing import Optional


def say_hello(name: Optional[str] = None) -> str:
    """Provides a simple greeting. If a name is provided, it will be used.

    Args:
        name (str, optional): The name of the person to greet. Defaults to a generic greeting if not provided.

    Returns:
        str: A friendly greeting message.
    """
    if name:
        greeting = f"Hello, {name}!"
        print(f"--- Tool: say_hello called with name: {name} ---")
    else:
        greeting = "Hello there!" # Default greeting if name is None or not explicitly passed
        print(f"--- Tool: say_hello called without a specific name (name_arg_value: {name}) ---")
    return greeting

def say_goodbye() -> str:
    """Provides a simple farewell message to conclude the conversation."""
    print(f"--- Tool: say_goodbye called ---")
    return "Goodbye! Have a great day."

print("Greeting and Farewell tools defined.")

# Optional self-test
print(say_hello("Alice"))
print(say_hello()) # Test with no argument (should use default "Hello there!")
print(say_hello(name=None)) # Test with name explicitly as None (should use default "Hello there!")

Greeting and Farewell tools defined.
--- Tool: say_hello called with name: Alice ---
Hello, Alice!
--- Tool: say_hello called without a specific name (name_arg_value: None) ---
Hello there!
--- Tool: say_hello called without a specific name (name_arg_value: None) ---
Hello there!


In [10]:
# @title Definindo sub-agentes (chatgpt)

# If you want to use models other than Gemini, Ensure LiteLlm is imported and API keys are set (from Step 0/2)
# from google.adk.models.lite_llm import LiteLlm

# --- Greeting Agent ---
greeting_agent = None
try:
    greeting_agent = Agent(
        #model = MODEL_GEMINI_2_5_FLASH,
        model=LiteLlm(model=MODEL_GPT_4O),
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                    "Use the 'say_hello' tool to generate the greeting. "
                    "If the user provides their name, make sure to pass it to the tool. "
                    "Do not engage in any other conversation or tasks.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.", # Crucial for delegation
        tools=[say_hello],
    )
    print(f"✅ Agente '{greeting_agent.name}' criado usando o modelo '{greeting_agent.model}'.")
except Exception as e:
    print(f"❌ Não foi possível criar o sub-agente 1, verifique a chave de API ({greeting_agent.model}). Error: {e}")

# --- Farewell Agent ---
farewell_agent = None
try:
    farewell_agent = Agent(
        #model = MODEL_GEMINI_2_5_FLASH,
        model=LiteLlm(model=MODEL_GPT_4O),
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                    "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                    "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                    "Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # Crucial for delegation
        tools=[say_goodbye],
    )
    print(f"✅ Agente '{farewell_agent.name}' criado usando o modelo '{farewell_agent.model}'.")
except Exception as e:
    print(f"❌ Não foi possível criar o sub-agente 1, verifique a chave de API ({farewell_agent.model}). Error: {e}")

✅ Agente 'greeting_agent' criado usando o modelo 'model='openai/gpt-4.1-nano' llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x7a84b2a1ea20>'.
✅ Agente 'farewell_agent' criado usando o modelo 'model='openai/gpt-4.1-nano' llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x7a84bf4a9670>'.


In [11]:
# @title Definindo Agente Orquestrador (Claude)

root_agent = None
runner_root = None # Initialize runner

if greeting_agent and farewell_agent and 'get_weather_open_meteo' in globals():
    root_agent_model = MODEL_CLAUDE_SONNET

    weather_agent_team = Agent(
        name="agente_meteorologico_v2",
        model=root_agent_model,
        description="The main coordinator agent. Handles weather requests and delegates greetings/farewells to specialists.",
        instruction="You are the main Weather Agent coordinating a team. Your primary responsibility is to provide weather information. "
                    "Use the 'get_weather' tool ONLY for specific weather requests (e.g., 'weather in London'). "
                    "You have specialized sub-agents: "
                    "1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. "
                    "2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. "
                    "Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. "
                    "If it's a weather request, handle it yourself using 'get_weather'. "
                    "For anything else, respond appropriately or state you cannot handle it.",
        tools=[get_weather_open_meteo],
        # ALTERAÇÃO: adicionando sub-agentes
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ Agente Orquestrador (root) '{weather_agent_team.name}' \nCriado usando o modelo: '{root_agent_model}'\nCom sub-agentes: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ Não foi possível criar o agente orquestrador porque um ou mais subagentes não foram inicializados ou a ferramenta 'get_weather' está ausente.")
    if not greeting_agent: print(" - Sub-agente 1 se perdeu.")
    if not farewell_agent: print(" - Sub-agente 2 se perdeu.")
    if 'get_weather_open_meteo' not in globals(): print(" - Ferramenta perdida.")

✅ Agente Orquestrador (root) 'agente_meteorologico_v2' 
Criado usando o modelo: 'anthropic/claude-sonnet-4-5-20250929'
Com sub-agentes: ['greeting_agent', 'farewell_agent']


In [ ]:
# @title Interagindo com o Time de Agentes

import asyncio

root_agent_var_name = 'root_agent' # Default name from Step 3 guide
if 'weather_agent_team' in globals(): # Check if user used this name instead
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'weather_agent_team') not found. Cannot define run_team_conversation.")
    root_agent = None

if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_team_conversation():
        print("\n--- Testando Delegação de Sub-Agentes no Time ---")
        session_service = InMemorySessionService()
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team"
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )
        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]
        runner_agent_team = Runner( # Or use InMemoryRunner
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"Runner created for agent '{actual_root_agent.name}'.")

        # --- Interactions using await (correct within async def) ---
        await call_agent_async(query = "Hello there!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "What is the weather in New York?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Thanks, bye!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    print("Attempting execution using 'await' (default for notebooks)...")
    await run_team_conversation()

else:
    print("\n⚠️ Skipping agent team conversation execution as the root agent was not successfully defined in a previous step.")

Attempting execution using 'await' (default for notebooks)...

--- Testando Delegação de Sub-Agentes no Time ---
Session created: App='weather_tutorial_agent_team', User='user_1_agent_team', Session='session_001_agent_team'
Runner created for agent 'agente_meteorologico_v2'.

>>> Questão do usuário: Hello there!
--- Tool: say_hello called without a specific name (name_arg_value: None) ---
<<< Resposta do Agente: Hello there!

>>> Questão do usuário: What is the weather in New York?
<<< Resposta do Agente: The current weather in New York shows:
- **Temperature**: 10.2°C (50.4°F)
- **Humidity**: 45%
- **Wind Speed**: 21.4 km/h (13.3 mph)

It's a cool day with moderate winds and relatively low humidity. You might want to bring a light jacket if you're heading out!

>>> Questão do usuário: Thanks, bye!
--- Tool: say_goodbye called ---
<<< Resposta do Agente: Goodbye! Have a great day.


## Adicionando Memória e Personalização com o Session State

In [17]:
# @title 1. Inicializar novo session service e state

from google.adk.sessions import InMemorySessionService

# nova instância de session service
session_service_stateful = InMemorySessionService()
print("✅ Novo InMemorySessionService criado.")

# credenciais da nova sessão
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# Define initial state data - user prefers Celsius initially
initial_state = {
    "user_preference_temperature_unit": "Celsius"
}

# Cria a sessão fornecendo um estado inicial
session_stateful = await session_service_stateful.create_session(
    app_name=APP_NAME, # Use the consistent app name
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state # <<< Initialize state during creation
)
print(f"✅ Sessão '{SESSION_ID_STATEFUL}' criada pelo usuário '{USER_ID_STATEFUL}'.")

# Verifica inicialização do state
retrieved_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id = SESSION_ID_STATEFUL)
print("\n--- Initial Session State ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("Error: Could not retrieve session.")

✅ Novo InMemorySessionService criado.
✅ Sessão 'session_state_demo_001' criada pelo usuário 'user_state_demo'.

--- Initial Session State ---
{'user_preference_temperature_unit': 'Celsius'}


In [27]:
# @title 2. Atualização da ferramenta meteorologica usando API (open-meteo)

# @title Ferramenta do Agente com Memória (open-meteo API)
import requests
from google.adk.tools.tool_context import ToolContext

def get_weather_open_meteo(city_name: str, tool_context: ToolContext) -> str:
    """
    Busca o clima atual usando Open-Meteo, respeitando preferências de unidade
    e salvando o contexto da cidade.
    """
    print(f"--- Tool: get_weather_open_meteo chamada para: {city_name} ---")

    # 1. MEMÓRIA: Ler preferência do usuário (Padrão: Celsius)
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Celsius")
    print(f"--- Tool: Preferência de unidade detectada: {preferred_unit} ---")

    try:
        # Geocoding (Converter nome da cidade em Lat/Long)
        geo_url = "https://geocoding-api.open-meteo.com/v1/search"
        geo_params = {"name": city_name, "count": 1, "language": "pt", "format": "json"}

        geo_res = requests.get(geo_url, params=geo_params).json()

        if not geo_res.get("results"):
            return f"Não encontrei a localização: {city_name}"

        location = geo_res["results"][0]
        lat = location["latitude"]
        long = location["longitude"]
        real_name = location["name"]

        # Busca Meteorologica
        meteo_url = "https://api.open-meteo.com/v1/forecast"
        meteo_params = {
            "latitude": lat,
            "longitude": long,
            "current": "temperature_2m,relative_humidity_2m,wind_speed_10m"
        }

        meteo_res = requests.get(meteo_url, params=meteo_params).json()
        current = meteo_res.get("current", {})

        # Dados brutos (Open-Meteo sempre retorna Celsius por padrão nesta chamada)
        temp_value = current.get('temperature_2m')
        hum = current.get('relative_humidity_2m')
        wind = current.get('wind_speed_10m')
        unit_label = "°C"

        # 2. LÓGICA DE ESTADO: Converter se necessário
        if preferred_unit == "Fahrenheit" and temp_value is not None:
            temp_value = (temp_value * 9/5) + 32
            unit_label = "°F"

        # 3. MEMÓRIA: Salvar a cidade consultada no estado
        # Isso permite que a próxima pergunta seja "E qual a população de lá?" sem citar a cidade
        tool_context.state["last_city_checked"] = real_name
        print(f"--- Tool: Estado atualizado. 'last_city_checked': {real_name} ---")

        return (f"Previsão para {real_name}: "
                f"Temperatura de {temp_value:.1f}{unit_label}, "
                f"Umidade de {hum}%, "
                f"Vento a {wind} km/h.")

    except Exception as e:
        return f"Erro ao processar o pedido: {e}"

print("✅ Nova ferramenta definida.")

✅ Nova ferramenta definida.


In [ ]:
# @title Classe simples para simular o contexto do agente
class MockToolContext:
    def __init__(self, initial_state=None):
        self.state = initial_state or {}

# Teste 1: Padrão (Celsius)
ctx_celsius = MockToolContext(initial_state={})
print(get_weather_open_meteo("Palmas", ctx_celsius))
print(f"Memória após execução: {ctx_celsius.state}")

print("-" * 30)

# Teste 2: Com preferência de usuário (Fahrenheit)
ctx_fahrenheit = MockToolContext(initial_state={"user_preference_temperature_unit": "Fahrenheit"})
print(get_weather_open_meteo("Palmas", ctx_fahrenheit))

--- Tool: get_weather_open_meteo chamada para: Palmas ---
--- Tool: Preferência de unidade detectada: Celsius ---
--- Tool: Estado atualizado. 'last_city_checked': Palmas ---
Previsão para Palmas: Temperatura de 25.4°C, Umidade de 87%, Vento a 2.1 km/h.
Memória após execução: {'last_city_checked': 'Palmas'}
------------------------------
--- Tool: get_weather_open_meteo chamada para: Palmas ---
--- Tool: Preferência de unidade detectada: Fahrenheit ---
--- Tool: Estado atualizado. 'last_city_checked': Palmas ---
Previsão para Palmas: Temperatura de 77.7°F, Umidade de 87%, Vento a 2.1 km/h.


In [19]:
# @title 3. Redefinindo sub-agentes e atualizando o agente orquestrador

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner

# --- Sub-agentes

greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GPT_4O,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Error: {e}")

farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GPT_4O,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Error: {e}")

# --- Agente Orquestrador

root_agent_stateful = None
runner_root_stateful = None #

if greeting_agent and farewell_agent and 'get_weather_open_meteo' in globals():

    root_agent_model = MODEL_CLAUDE_SONNET

    root_agent_stateful = Agent(
        name="agente_meteorologico_estado_persistente",
        model=root_agent_model,
        description="Main agent: Provides weather (state-aware unit), delegates greetings/farewells, saves report to state.",
        instruction="You are the main Weather Agent. Your job is to provide weather using 'get_weather_stateful'. "
                    "The tool will format the temperature based on user preference stored in state. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather_open_meteo], # seleciona ferramentas
        sub_agents=[greeting_agent, farewell_agent], # inclui sub-agentes
        output_key="last_weather_report" # <<< salva automaticamente a resposta final do agente
    )
    print(f"✅ Root Agent '{root_agent_stateful.name}' created using stateful tool and output_key.")

    # Cria Runner e um novo Session Service
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful # inicia um novo estado session service
    )
    print(f"✅ Runner criado para agente raiz com estado '{runner_root_stateful.agent.name}' usando session service com estado.")

else:
    print("❌ Não foi possível criar o agente root.")
    if not greeting_agent: print(" - greeting_agent função perdida.")
    if not farewell_agent: print(" - farewell_agent função perdida.")
    if 'get_weather_open_meteo' not in globals(): print(" - get_weather_open_meteo tool missing.")

✅ Agent 'greeting_agent' redefined.
✅ Agent 'farewell_agent' redefined.
✅ Root Agent 'agente_meteorologico_estado_persistente' created using stateful tool and output_key.
✅ Runner criado para agente raiz com estado 'agente_meteorologico_estado_persistente' usando session service com estado.


In [ ]:
# @title 4. Interagindo e testando o fluxo

import asyncio

if 'runner_root_stateful' in globals() and runner_root_stateful:
    async def run_stateful_conversation():
        print("\n--- Testando o estado de memória: Conversão de Unidade de Temperatura & output_key ---")

        # 1. Check weather (Uses initial state: Celsius)
        print("--- Turno 1: Solicitando o clima em Londres (suposto em Celsius) ---")
        await call_agent_async(query= "Qual o clima em Londres?",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

        # 2. Atualizando manualmente o estado de memória para Fahrenheit
        print("\n--- Atualizando estado para Fahrenheit ---")
        try:
            stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
            stored_session.state["user_preference_temperature_unit"] = "Fahrenheit"
            # Optional: You might want to update the timestamp as well if any logic depends on it
            # import time
            # stored_session.last_update_time = time.time()
            print(f"--- Atualização de session state. Atual 'user_preference_temperature_unit': {stored_session.state.get('user_preference_temperature_unit', 'Not Set')} ---")
        except KeyError:
            print(f"--- Error: Não foi possível recuperar a sessão '{SESSION_ID_STATEFUL}' do armazenamento interno do usuário '{USER_ID_STATEFUL}' no app '{APP_NAME}' para a atualizar o estado de memória.")
        except Exception as e:
             print(f"--- Erro interno na atualização do session state: {e} ---")

        # 3. Checando novamente o clima (A ferramenta agora deve usar Fahrenheit)
        # Também irá atualizar 'last_weather_report' via output_key
        print("\n--- Turno 2: Solicitando o clima em Londres (suposto em Fahrenheit) ---")
        await call_agent_async(query= "Me conta como que tá o clima la em Londres",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

        # 4. Testando delegação para sub-agentes
        # Essa checagem irá atualizar 'last_weather_report' novamente, sobrescrevendo o relatorio sobre o clima em maringá
        print("\n--- Turno 3: Saudações ao nobre guerreirinho cibernético ---")
        await call_agent_async(query= "Olá meu nobre!",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

    print("Attempting execution using 'await' (default for notebooks)...")
    await run_stateful_conversation()

    # METHOD 2: asyncio.run (For Standard Python Scripts [.py])
    # If running this code as a standard Python script from your terminal,
    # the script context is synchronous. `asyncio.run()` is needed to
    # create and manage an event loop to execute your async function.
    # To use this method:
    # 1. Comment out the `await run_stateful_conversation()` line above.
    # 2. Uncomment the following block:
    """
    import asyncio
    if __name__ == "__main__": # Ensures this runs only when script is executed directly
        print("Executing using 'asyncio.run()' (for standard Python scripts)...")
        try:
            # This creates an event loop, runs your async function, and closes the loop.
            asyncio.run(run_stateful_conversation())
        except Exception as e:
            print(f"An error occurred: {e}")
    """

    # --- Verificando valor final de session state após a conversa ---
    print("\n--- Verificando Valor Final de Session State ---")
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id= USER_ID_STATEFUL,
                                                         session_id=SESSION_ID_STATEFUL)
    if final_session:
        # Use .get() para acessar de forma segura potenciais chaves perdidas
        print(f"Unidade de Preferência ao Final: {final_session.state.get('user_preference_temperature_unit', 'Not Set')}")
        print(f"Último relatório climático (output_key): {final_session.state.get('last_weather_report', 'Not Set')}")
        print(f"Última cidade verificada (tool): {final_session.state.get('last_city_checked_stateful', 'Not Set')}")
        # Print full state for detailed view
        # print(f"Full State Dict: {final_session.state}") # For detailed view
    else:
        print("\n❌ Error: Não foi possível recuperar session state.")

else:
    print("\n⚠️ Skipping state test conversation. Stateful root agent runner ('runner_root_stateful') is not available.")

Attempting execution using 'await' (default for notebooks)...

--- Testando o estado de memória: Conversão de Unidade de Temperatura & output_key ---
--- Turno 1: Solicitando o clima em Londres (suposto em Celsius) ---

>>> Questão do usuário: Qual o clima em Londres?
--- Tool: get_weather_open_meteo chamada para: Londres ---
--- Tool: Preferência de unidade detectada: Fahrenheit ---
--- Tool: Estado atualizado. 'last_city_checked': Londres ---
<<< Resposta do Agente: A previsão do tempo para **Londres** é:
- 🌡️ **Temperatura**: 42.6°F (aproximadamente 5.9°C)
- 💧 **Umidade**: 85%
- 💨 **Vento**: 11.4 km/h

O clima em Londres está bem frio! A temperatura está em torno de 6°C, típico de um dia de inverno ou outono. A umidade está alta (85%), e há um vento moderado. É dia de agasalho por lá! 🧥🌧️

--- Atualizando estado para Fahrenheit ---
--- Atualização de session state. Atual 'user_preference_temperature_unit': Fahrenheit ---

--- Turno 2: Solicitando o clima em Londres (suposto em Fahre

## Adicionando Segurança - Proteção na Entrada do Modelo Usando Callback (before_model_callback)

In [29]:
# @title 1. Definindo a função de proteção before_model_callback

from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types # For creating response content
from typing import Optional

def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    Inspects the latest user message for 'BLOCK'. If found, blocks the LLM call
    and returns a predefined LlmResponse. Otherwise, returns None to proceed.
    """
    agent_name = callback_context.agent_name # Get the name of the agent whose model call is being intercepted
    print(f"--- Callback: block_keyword_guardrail running for agent: {agent_name} ---")

    # Extract the text from the latest user message in the request history
    last_user_message_text = ""
    if llm_request.contents:
        # Find the most recent message with role 'user'
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
                # Assuming text is in the first part for simplicity
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break # Found the last user message text

    print(f"--- Callback: Inspecting last user message: '{last_user_message_text[:100]}...' ---") # Log first 100 chars

    # --- Guardrail Logic ---
    keyword_to_block = "BLOCK"
    if keyword_to_block in last_user_message_text.upper(): # Case-insensitive check
        print(f"--- Callback: Found '{keyword_to_block}'. Blocking LLM call! ---")
        # Optionally, set a flag in state to record the block event
        callback_context.state["guardrail_block_keyword_triggered"] = True
        print(f"--- Callback: Set state 'guardrail_block_keyword_triggered': True ---")

        # Construct and return an LlmResponse to stop the flow and send this back instead
        return LlmResponse(
            content=types.Content(
                role="model", # Mimic a response from the agent's perspective
                parts=[types.Part(text=f"I cannot process this request because it contains the blocked keyword '{keyword_to_block}'.")],
            )
            # Note: You could also set an error_message field here if needed
        )
    else:
        # Keyword not found, allow the request to proceed to the LLM
        print(f"--- Callback: Keyword not found. Allowing LLM call for {agent_name}. ---")
        return None # Returning None signals ADK to continue normally

print("✅ block_keyword_guardrail function defined.")

✅ block_keyword_guardrail function defined.


In [34]:
# @title 2. Redefinindo sub-agentes e atualizando o agente orquestrador (proteção callback)

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner

# --- Sub-agentes

greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GPT_4O,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Error: {e}")

farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GPT_4O,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Error: {e}")

# --- Agente Orquestrador

root_agent_model_guardrail = None
runner_root_model_guardrail = None

if greeting_agent and farewell_agent and 'get_weather_open_meteo' in globals():

    root_agent_model = MODEL_CLAUDE_SONNET

    root_agent_model_guardrail = Agent(
        name="agente_meteorologico_protecao_callback",
        model=root_agent_model,
        description="Main agent: Provides weather (state-aware unit), delegates greetings/farewells, saves report to state.",
        instruction="You are the main Weather Agent. Your job is to provide weather using 'get_weather_stateful'. "
                    "The tool will format the temperature based on user preference stored in state. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather_open_meteo], # seleciona ferramentas
        sub_agents=[greeting_agent, farewell_agent], # inclui sub-agentes
        output_key="last_weather_report", # salva automaticamente a resposta final do agente
        before_model_callback=block_keyword_guardrail # <<< adiciona a função de callback ao agente root
    )
    print(f"✅ Root Agent '{root_agent_model_guardrail.name}' created using stateful tool and output_key.")

    # Cria Runner novo e mantem o estato da sessão anterior (mesmo Session Service)
    if 'session_service_stateful' in globals():
        runner_root_model_guardrail = Runner(
            agent=root_agent_model_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # herda o mesmo session service do agente anterior
        )
        print(f"✅ Runner criado para agente root com estado '{runner_root_model_guardrail.agent.name}' usando session service com estado.")
    else:
        print("❌ Não foi possível criar o runner. 'session_service_stateful'. Runner perdido.")


else:
    print("❌ Não foi possível criar o agente root.")
    if not greeting_agent: print(" - greeting_agent função perdida.")
    if not farewell_agent: print(" - farewell_agent função perdida.")
    if 'get_weather_open_meteo' not in globals(): print(" - get_weather_open_meteo tool missing.")

✅ Agent 'greeting_agent' redefined.
✅ Agent 'farewell_agent' redefined.
✅ Root Agent 'agente_meteorologico_protecao_callback' created using stateful tool and output_key.
✅ Runner criado para agente root com estado 'agente_meteorologico_protecao_callback' usando session service com estado.


In [ ]:
# @title 3. Testando o Modelo com Proteção Callback
import asyncio

if 'runner_root_model_guardrail' in globals() and runner_root_model_guardrail:
    # Define the main async function for the guardrail test conversation.
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_guardrail_test_conversation():
        print("\n--- Testing Model Input Guardrail ---")

        # Use the runner for the agent with the callback and the existing stateful session ID
        # Define a helper lambda for cleaner interaction calls
        interaction_func = lambda query: call_agent_async(query,
                                                         runner_root_model_guardrail,
                                                         USER_ID_STATEFUL, # Use existing user ID
                                                         SESSION_ID_STATEFUL # Use existing session ID
                                                        )
        # 1. Normal request (Callback allows, should use Fahrenheit from previous state change)
        print("--- Turno 1: Qual o clima em Niterói (esperado em Fahrenheit) ---")
        await interaction_func("Qual o clima em Niterói?")

        # 2. Request containing the blocked keyword (Callback intercepts)
        print("\n--- Turno 2: Verificando bloqueio de palavras chave (esperado bloqueio do prompt) ---")
        await interaction_func("BLOCK o prompt sobre o tempo em Londrina") # Callback dever capturar BLOCK

        # 3. Saudações (Callback deve permitir ao root delegar ações)
        print("\n--- Turno 3: Enviando saudações a Sr. Robô (deve ser permitido) ---")
        await interaction_func("Olá novamente meu amigo cibernético.")


    print("Executando com 'await' (padrão para notebooks)...")
    await run_guardrail_test_conversation()


    print("\n--- Inspecting Final Session State (After Guardrail Test) ---")
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id=SESSION_ID_STATEFUL)
    if final_session:
        print(f"Flag acionada pela proteção: {final_session.state.get('guardrail_block_keyword_triggered', 'Not Set (or False)')}")
        print(f"Último relatório climático: {final_session.state.get('last_weather_report', 'Not Set')}") # Should be London weather if successful
        print(f"Unidade de Temperatura: {final_session.state.get('user_preference_temperature_unit', 'Not Set')}") # Should be Fahrenheit
        print(f"Full State Dict: {final_session.state}") # For detailed view
    else:
        print("\n❌ Error: Could not retrieve final session state.")

else:
    print("\n⚠️ Skipping model guardrail test. Runner ('runner_root_model_guardrail') is not available.")

Executando com 'await' (padrão para notebooks)...

--- Testing Model Input Guardrail ---
--- Turno 1: Qual o clima em Niterói (esperado em Fahrenheit) ---

>>> Questão do usuário: Qual o clima em Niterói?
--- Tool: say_hello called with name: Qual o clima em Niterói? ---
<<< Resposta do Agente: Olá! Atualmente não tenho informações específicas sobre o clima em Niterói. Gostaria que eu tentasse obter essas informações para você?

--- Turno 2: Verificando bloqueio de palavras chave (esperado bloqueio do prompt) ---

>>> Questão do usuário: BLOCK o prompt sobre o tempo em Londrina
<<< Resposta do Agente: Entendido! Vou bloquear o prompt sobre o tempo em Londrina.

--- Turno 3: Enviando saudações a Sr. Robô (deve ser permitido) ---

>>> Questão do usuário: Olá novamente meu amigo cibernético.
--- Tool: say_hello called with name: Olá novamente meu amigo cibernético. ---
<<< Resposta do Agente: Hello, Olá novamente meu amigo cibernético.!

--- Inspecting Final Session State (After Guardrail

## Adicionando Segurança - Proteção para Ferramentas Usando Callback (before_tool_callback)

In [48]:
# @title 1. Definindo a função de proteção das ferramentas

from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Optional, Dict, Any

def block_city_tool_guardrail(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """
    Checks if 'get_weather_stateful' is called for 'Paris'.
    If so, blocks the tool execution and returns a specific error dictionary.
    Otherwise, allows the tool call to proceed by returning None.
    """
    tool_name = tool.name
    agent_name = tool_context.agent_name
    print(f"--- Callback: block_city_tool_guardrail sendo executada '{tool_name}' pelo agente '{agent_name}' ---")
    print(f"--- Callback: Verificando args: {args} ---")

    # --- Guardrail Logic ---
    target_tool_name = "get_weather_open_meteo"
    blocked_city = "xique-xique" # Garante que a cidade bloqueada é sempre em minúsculas para comparação

    if tool_name == target_tool_name:
        # CORREÇÃO: Usar 'city_name' ao invés de 'city'
        city_argument = args.get("city_name", "") # Obtendo o argumento de forma segura
        if city_argument and city_argument.lower() == blocked_city:
            print(f"--- Callback: Detectada busca pela cidade proibida : '{city_argument}'. Ferramenta bloqueda! ---")
            # Optionally update state
            tool_context.state["guardrail_tool_block_triggered"] = True
            print(f"--- Callback: Estado atual 'guardrail_tool_block_triggered': True ---")

            return {
                "status": "error",
                "error_message": f"Policy restriction: Weather checks for '{city_argument.capitalize()}' are currently disabled by a tool guardrail."
            }
        else:
             # Corrige a declaração de impressão para mostrar o argumento real da cidade
             print(f"--- Callback: Cidade '{city_argument}' possui permissão de uso da ferramenta '{tool_name}'. ---")
    else:
        print(f"--- Callback: Ferramenta '{tool_name}' não foi utilizada. ---")


    print(f"--- Callback: Ferramenta permitida '{tool_name}'. ---")
    return None

print("✅ block_city_tool_guardrail function defined.")

✅ block_city_tool_guardrail function defined.


In [46]:
# @title 2. Redefinindo sub-agentes e atualizando o agente orquestrador (proteção callback)

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner

# --- Sub-agentes

greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GPT_4O,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Agente '{greeting_agent.name}' redefinido.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Error: {e}")

farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GPT_4O,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Agente '{farewell_agent.name}' redefinido.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Error: {e}")

# --- Agente Orquestrador

root_agent_tool_guardrail = None
runner_root_tool_guardrail = None

if ('greeting_agent' in globals() and greeting_agent and
    'farewell_agent' in globals() and farewell_agent and
    'get_weather_open_meteo' in globals() and
    'block_keyword_guardrail' in globals() and
    'block_city_tool_guardrail' in globals()):

    root_agent_model = MODEL_CLAUDE_SONNET

    root_agent_tool_guardrail = Agent(
        name="agente_meteorologico_duas_protecoes_callback",
        model=root_agent_model,
        description="Main agent: Provides weather (state-aware unit), delegates greetings/farewells, saves report to state.",
        instruction="You are the main Weather Agent. Your job is to provide weather using 'get_weather_stateful'. "
                    "The tool will format the temperature based on user preference stored in state. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather_open_meteo], # seleciona ferramentas
        sub_agents=[greeting_agent, farewell_agent], # inclui sub-agentes
        output_key="last_weather_report", # salva automaticamente a resposta final do agente
        before_model_callback=block_keyword_guardrail, # <<< adiciona a função de callback ao agente root
        before_tool_callback=block_city_tool_guardrail # <<< adiciona a função de callback ao agente root
    )
    print(f"✅ Agente Root '{root_agent_tool_guardrail.name}' criado.")

    # Cria Runner novo e mantem o estato da sessão anterior (mesmo Session Service)
    if 'session_service_stateful' in globals():
        runner_root_tool_guardrail = Runner(
            agent=root_agent_tool_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # herda o mesmo session service do agente anterior
        )
        print(f"✅ Runner criado para agente root com estado '{runner_root_tool_guardrail.agent.name}' usando session service com estado.")
    else:
        print("❌ Não foi possível criar o runner. 'session_service_stateful'. Runner perdido.")


else:
    print("❌ Não foi possível criar o agente root.")
    if not greeting_agent: print(" - greeting_agent função perdida.")
    if not farewell_agent: print(" - farewell_agent função perdida.")
    if 'get_weather_open_meteo' not in globals(): print(" - get_weather_open_meteo tool missing.")

✅ Agente 'greeting_agent' redefinido.
✅ Agente 'farewell_agent' redefinido.
✅ Agente Root 'agente_meteorologico_duas_protecoes_callback' criado.
✅ Runner criado para agente root com estado 'agente_meteorologico_duas_protecoes_callback' usando session service com estado.


In [49]:
# @title 3. Interagindo com as novas ferramentas
import asyncio

if 'runner_root_tool_guardrail' in globals() and runner_root_tool_guardrail:
    async def run_tool_guardrail_test():
        print("\n--- Testando ferramenta de proteção dos argumentos ('xique-xique' é a cidade proibida) ---")

        interaction_func = lambda query: call_agent_async(query,
                                                         runner_root_tool_guardrail,
                                                         USER_ID_STATEFUL, # Use existing user ID
                                                         SESSION_ID_STATEFUL # Use existing session ID
                                                        )
        print("--- Turno 1: Clima em Curitiba (permitido) ---")
        await interaction_func("Como está o clima em curitiba?")

        print("\n--- Turno 2: Clima em Xique-xique (a cidade proibida) ---")
        await interaction_func("E em xique-xique?")

        print("\n--- Turno 3: Clima em Brasília (permitido novamente) ---")
        await interaction_func("Conte-me sobre o clima de Brasília hoje.")

    print("Attempting execution using 'await' (default for notebooks)...")
    await run_tool_guardrail_test()

    print("\n--- Inspecting Final Session State (After Tool Guardrail Test) ---")
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id= SESSION_ID_STATEFUL)
    if final_session:
        print(f"Flag de Ativação da Função de Callback: {final_session.state.get('guardrail_tool_block_triggered', 'Not Set (or False)')}")
        print(f"Último relatório climático: {final_session.state.get('last_weather_report', 'Not Set')}")
        print(f"Unidade de Temperatura: {final_session.state.get('user_preference_temperature_unit', 'Not Set')}")
        print(f"Dict de Estado: {final_session.state}")
    else:
        print("\n❌ Error: Não foi possível recuperar o session state final.")

else:
    print("\n⚠️ Pulando teste da ferramenta. executor ('runner_root_tool_guardrail') não está disponível.")

Attempting execution using 'await' (default for notebooks)...

--- Testando ferramenta de proteção dos argumentos ('xique-xique' é a cidade proibida) ---
--- Turno 1: Clima em Curitiba (permitido) ---

>>> Questão do usuário: Como está o clima em curitiba?
--- Callback: block_keyword_guardrail running for agent: agente_meteorologico_duas_protecoes_callback ---
--- Callback: Inspecting last user message: 'Como está o clima em curitiba?...' ---
--- Callback: Keyword not found. Allowing LLM call for agente_meteorologico_duas_protecoes_callback. ---
--- Callback: block_city_tool_guardrail sendo executada 'get_weather_open_meteo' pelo agente 'agente_meteorologico_duas_protecoes_callback' ---
--- Callback: Verificando args: {'city_name': 'Curitiba'} ---
--- Callback: Cidade 'Curitiba' possui permissão de uso da ferramenta 'get_weather_open_meteo'. ---
--- Callback: Ferramenta permitida 'get_weather_open_meteo'. ---
--- Tool: get_weather_open_meteo chamada para: Curitiba ---
--- Tool: Preferê